# Описание проекта

Цель иследования: проанализировать базу данных. В ней — информация о книгах, издательствах, авторах, а также пользовательские обзоры книг. Эти данные помогут сформулировать ценностное предложение для нового продукта.

Задачи:
1. Посчитаnm, сколько книг вышло после 1 января 2000 года;
2. Для каждой книги посчитайте количество обзоров и среднюю оценку;
3. Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры;
4. Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками;
5. Посчитайте среднее количество обзоров от пользователей, которые поставили больше 50 оценок.

# Иследование данных

In [1]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import create_engine 

In [2]:
# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db'} # название базы данных

connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
 db_config['pwd'],
 db_config['host'],
 db_config['port'],
 db_config['db'])

# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'})

In [3]:
# Выведим по строке в каждой таблицы
query = ''' 
SELECT *
FROM books
LIMIT 1
'''
display(pd.io.sql.read_sql(query, con = engine))

query = ''' 
SELECT *
FROM authors
LIMIT 1
'''
display(pd.io.sql.read_sql(query, con = engine))

query = ''' 
SELECT *
FROM ratings
LIMIT 1
'''
display(pd.io.sql.read_sql(query, con = engine))

query = ''' 
SELECT *
FROM reviews
LIMIT 1
'''
display(pd.io.sql.read_sql(query, con = engine))

query = ''' 
SELECT *
FROM publishers
LIMIT 1
'''
display(pd.io.sql.read_sql(query, con = engine))

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93


,author_id,author
0,1,A.S. Byatt


,rating_id,book_id,username,rating
0,1,1,ryanfranco,4


,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...


,publisher_id,publisher
0,1,Ace


# Анализ базы данных

#### Посчитаем, сколько книг вышло после 1 января 2000 года

In [4]:
query = ''' 
SELECT COUNT(title)
FROM books
WHERE CAST(publication_date AS date) >= '2000-01-01'
'''
pd.io.sql.read_sql(query, con = engine)

,count
0,821


- После 1 января 2000 года вышла 821 книга

#### Для каждой книги посчитайем количество обзоров и среднюю оценку

In [5]:
query = ''' 
WITH 
ar AS (SELECT book_id,
            AVG(rating) AS avg_rating
        FROM ratings
        GROUP BY book_id),
cr AS (SELECT book_id,
                    COUNT(review_id) AS count_review
                FROM reviews
                GROUP BY book_id
                ORDER BY book_id)
                
SELECT b.title,
        ar.avg_rating,
        cr.count_review
FROM ar
FULL OUTER JOIN books AS b ON ar.book_id = b.book_id
FULL OUTER JOIN cr ON cr.book_id = b.book_id


'''
pd.io.sql.read_sql(query, con = engine)

,title,avg_rating,count_review
0,'Salem's Lot,3.666667,2.0
1,1 000 Places to See Before You Die,2.500000,1.0
2,13 Little Blue Envelopes (Little Blue Envelope...,4.666667,3.0
3,1491: New Revelations of the Americas Before C...,4.500000,2.0
4,1776,4.000000,4.0
...,...,...,...
995,The Cat in the Hat and Other Dr. Seuss Favorites,5.000000,NaN
996,Anne Rice's The Vampire Lestat: A Graphic Novel,3.666667,NaN
997,Essential Tales and Poems,4.000000,NaN
998,Leonardo's Notebooks,4.000000,NaN


- Для 6 книг отсутствуют рецензии, рейтинг присутсвует у всех книг

#### Определим издательство, которое выпустило наибольшее число книг толще 50 страниц (исключим из анализа брошюры)

In [6]:
query = ''' 
SELECT publisher,
    COUNT(title) AS count_title
FROM books AS b
LEFT JOIN publishers AS p ON b.publisher_id = p.publisher_id
WHERE num_pages > 50
GROUP BY publisher
ORDER BY count_title DESC
LIMIT 1
'''
pd.io.sql.read_sql(query, con = engine)

,publisher,count_title
0,Penguin Books,42


- Издательство выпустившее наибольшее количество книг (с количеством страниц больше 50) Penguin Books

#### Определим автора с самой высокой средней оценкой книг — учитывая только книги с 50 и более оценками;

In [11]:
query = ''' 
WITH 
ar AS (SELECT book_id,
            COUNT(rating) AS count_rating,
            AVG(rating) AS avg_rating
        FROM ratings
        GROUP BY book_id)
                
SELECT a.author,
        AVG(avg_rating) AS all_books_rating
FROM ar
FULL OUTER JOIN books AS b ON ar.book_id = b.book_id
FULL OUTER JOIN authors AS a ON a.author_id = b.author_id
WHERE count_rating > 50
GROUP BY author
ORDER BY all_books_rating DESC
LIMIT 1
'''
pd.io.sql.read_sql(query, con = engine)

,author,all_books_rating
0,J.K. Rowling/Mary GrandPré,4.283844


- Джуан Ролинг автор с самой высокой оценкой книг

#### Посчитаем среднее количество обзоров от пользователей, которые поставили больше 50 оценок

In [8]:
query = ''' 
WITH 
ar AS (SELECT book_id,
            COUNT(rating) AS count_rating,
            AVG(rating) AS avg_rating
        FROM ratings
        GROUP BY book_id)
                
SELECT *
FROM ar
FULL OUTER JOIN books AS b ON ar.book_id = b.book_id
FULL OUTER JOIN authors AS a ON a.author_id = b.author_id
WHERE count_rating > 50
LIMIT 1
'''
pd.io.sql.read_sql(query, con = engine)

,book_id,count_rating,avg_rating,book_id,author_id,title,num_pages,publication_date,publisher_id,author_id,author
0,75,84,3.678571,75,106,Angels & Demons (Robert Langdon #1),736,2006-04-01,229,106,Dan Brown


In [9]:
query = ''' 

SELECT AVG(count_review)
FROM (SELECT COUNT(text) AS count_review
      FROM reviews 
      WHERE username IN (SELECT username
                        FROM ratings
                        GROUP BY username
                        HAVING COUNT(username) > 50)
      GROUP BY username) AS cr

'''
pd.io.sql.read_sql(query, con = engine)

,avg
0,24.333333


- У пользователей, которые поставили больше 50 оценок в среднем по 24 обзора

# Вывод

- После 1 января 2000 года вышла 821 книга
- Для каждой книги посчитали количество обзоров и среднюю оценку. Для 6 книг отсутствуют рецензии, рейтинг присутсвует у всех книг
- Джуан Ролинг автор с самой высокой оценкой книг
- У пользователей, которые поставили больше 50 оценок в среднем по 24 обзора